# Graph Coloring

## Setting Up the Notebook

## 1. Defining Preprocessing

In [1]:
from collections import defaultdict, namedtuple
Graph = namedtuple('graph', ['mapping', 'edges', 'nodes'])
Solution = namedtuple('solution', ['edges', 'colored_nodes'])

class InputDataProcessor:
    @staticmethod
    def process(filename):
        with open(filename, 'r') as file:
            data = file.read().strip()
        graph = Graph(mapping=defaultdict(lambda: []), edges=set(), nodes=set())
        defaultdict(lambda: [])
        lines = data.split("\n")
        lines = lines[1:]
        for line in lines:
            n1, n2 = line.split(" ")
            try:
                n1, n2 = int(n1), int(n2)
            except:
                pass
            graph.mapping[n1] += [n2]
            graph.mapping[n2] += [n1]
            graph.nodes.update({n1, n2})
            graph.edges.add(tuple(sorted((n1, n2))))
        return graph

## 2. Defining the Coloring Model

In [2]:
import json
import os
from itertools import product
import numpy as np
import pulp

In [3]:
class ColorPicker:
    def __init__(self, graph, **kwargs):
        self.graph = graph
        self.max_degree = self._get_max_degree()
        print(f">>>> Max degree: {self.max_degree}")
        self.colors = list(range(self.max_degree + 1))
        self._build_model(**kwargs)


    def _get_max_degree(self):
        return max(len(neighbors) for neighbors in self.graph.mapping.values())

    def _build_model(self, **kwargs):
        self.problem = pulp.LpProblem(name="Colouring", sense=pulp.LpMinimize)
        # Step 1: Define Variables
        self.vars = self._create_variables()
        # Step 2: Build Constraints
        constraint_builder = ConstraintBuilder(self.problem, self.graph, self.colors, self.vars)
        self.problem = constraint_builder.build_constriaints(**kwargs)
        # Step 3: Set Objective
        self._set_objective()

    def _create_variables(self):
        print(">>>> Creating Variables")
        self.x = pulp.LpVariable.dicts(
            name="x",
            indices=product(self.graph.nodes, self.colors),
            lowBound=0,
            upBound=1,
            cat=pulp.LpBinary
        )
        self.z = pulp.LpVariable(name="z", lowBound=0, cat=pulp.LpInteger)
        vars = {"x": self.x, "z": self.z}
        return vars


    def _set_objective(self):
        print(">>>> Setting Objective")
        self.objective_parts = {"obj": self.z}
        self.problem.setObjective(pulp.lpSum(self.objective_parts.values()))


    def solve(self, solver_name="CBC", time_limit=None):
        print(f">>>> Solving system")
        os.environ["GRB_LICENSE_FILE"] = "gurobi.lic"
        threads = os.cpu_count()
        solvers = {
            "CBC": pulp.PULP_CBC_CMD(msg=False, threads=threads, timeLimit=time_limit, gapRel=1e-5), #logPath='./cbc.log'
            "GUROBI": pulp.GUROBI(msg=True, threads=threads, timeLimit=time_limit, gapRel=1e-5, MIPFocus=1),
        }
        solver = solvers[solver_name]
        self.problem.solve(solver)
        self.status = pulp.LpStatus[self.problem.status]
        print(f">>>> Status {self.status}")
        solution = self._format_solution()
        return solution

    def _format_solution(self):
        colored_nodes = dict()
        if self.status == "Optimal":
            for node in self.graph.nodes:
                colored_nodes[node] = max(
                    ((color, self.x[node, color].value()) for color in self.colors), key=lambda x: x[1]
                )[0]
        else:
            colored_nodes = {node: None for node in self.graph.nodes}
        solution = Solution(edges=self.graph.edges, colored_nodes=colored_nodes)
        return solution


### 2.1. Constraint Builder

In [4]:
class ConstraintBuilder:
    def __init__(self, problem, graph, colors, vars):
        self.problem = problem
        self.graph = graph
        self.colors = colors
        self.vars = vars

    
    def build_constriaints(self, **kwargs):
        print(f">>>> Building Constraints")
        self._add_every_node_has_color_constraint()
        
        if kwargs.get("tight_max", True):
            print(">>>>> using tight objective definition")
            self._add_obj_is_maximum_tight_constraint()
        else:
            print(">>>>> using loose objective definition")
            self._add_obj_is_maximum_loose_constraint()
        
        
        if kwargs.get("neighbors_constr_infeasible", False):
            self._add_no_neighbors_with_same_color_infeasible_constraint()
        else:
            self._add_no_neighbors_with_same_color_constraint()
        return self.problem


    def _add_every_node_has_color_constraint(self):
        x = self.vars['x']
        for node in self.graph.nodes:
            self.problem.addConstraint(
                pulp.lpSum(x[node, color] for color in self.colors) == 1,
                name=f"node_has_color_{node}"
            )

    def _add_obj_is_maximum_loose_constraint(self):
        x, z = self.vars['x'], self.vars['z']
        for node in self.graph.nodes:
            self.problem.addConstraint(
                z >= pulp.lpSum(color * x[node, color] for color in self.colors),
                name=f"obj_is_maximum_loose_constraint_{node}"
            )

    def _add_obj_is_maximum_tight_constraint(self):
        x, z = self.vars['x'], self.vars['z']
        for node, color in product(self.graph.nodes, self.colors):
            self.problem.addConstraint(
                z >= color * x[node, color],
                name=f"obj_is_maximum_tight_constraint_{node}_{color}"
            )

    def _add_no_neighbors_with_same_color_constraint(self):
        x = self.vars['x']
        for color in self.colors:
            for (node1, node2) in self.graph.edges:
                self.problem.addConstraint(
                    x[node1, color] + x[node2, color] <= 1,
                    name=f"no_neighbors_with_same_color_constraint_{color}_{node1}_{node2}"
                )
    
    def _add_no_neighbors_with_same_color_infeasible_constraint(self):
        x = self.vars['x']
        for color in self.colors:
            for node, neighbors in self.graph.mapping.items():             
                self.problem.addConstraint(
                    x[node, color] + pulp.lpSum(x[neighbor, color] for neighbor in neighbors) <= 1,
                    name=f"no_neighbors_with_same_color_infeasible_constraint{color}_{node}"
                )

## 3. Defining Preprocessing

In [5]:
from pyvis.network import Network
import matplotlib.pyplot as plt
from matplotlib.colors import rgb2hex

from numpy import linspace
import IPython

class OutputDataProcessor:
    @staticmethod
    def process(solution):
        g = Network(notebook=True)
        if all(color is not None for color in solution.colored_nodes.values()):
            cmap = plt.get_cmap("gist_rainbow")
            color_count = len(set(solution.colored_nodes.values()))
            rgb_colors = {c: cmap(x) for c, x in enumerate(linspace(0, 1-1/color_count, color_count))}
            g.add_nodes(nodes=list(solution.colored_nodes.keys()), color=[rgb2hex(rgb_colors[color_label]) for color_label in solution.colored_nodes.values()])
        else:
            g.add_nodes(nodes=list(solution.colored_nodes.keys()))
        g.add_edges(solution.edges)    
        g.set_options("""{"edges": {"color": {"inherit": false}}, "physics":{"maxVelocity": 15}}""")
        filename = "vis.html"
        g.show(name=filename)
        return g, filename

# 4. Running Some test cases

### Example 1: An Small Instance

In [6]:
graph = InputDataProcessor.process('data/gc_4_1')
color_picker = ColorPicker(graph)
solution = color_picker.solve(solver_name="CBC")
g, filename = OutputDataProcessor.process(solution)
print(f"Solution Time: {color_picker.problem.solutionTime:.3f}")
g.show(filename)
#IPython.display.HTML(filename=filename)

>>>> Max degree: 3
>>>> Creating Variables
>>>> Building Constraints
>>>>> using tight objective definition
>>>> Setting Objective
>>>> Solving system
Using license file gurobi.lic
Set parameter CSManager to value https://gurobi.dev.galileodigital.net
Set parameter CSAPIAccessID
Set parameter CSAPISecret
Compute Server job ID: 6bcaa94e-8a54-4e9f-9600-35fc7558e83e
Capacity available on 'vm-gurobi-dev-001:61000' - connecting...
Established HTTPS encrypted connection
Changed value of parameter threads to 12
   Prev: 0  Min: 0  Max: 1024  Default: 0
Changed value of parameter MIPFocus to 1
   Prev: 0  Min: 0  Max: 3  Default: 0
>>>> Status Optimal
Solution Time: 0.027


### Example 2: Tight V.S. Losse

In [7]:
graph = InputDataProcessor.process('data/gc_20_1')
color_picker = ColorPicker(graph, tight_max=False)
solution = color_picker.solve(solver_name="CBC")
g, filename = OutputDataProcessor.process(solution)
print(f"Solution Time: {color_picker.problem.solutionTime:.3f}")
g.show(filename)

>>>> Max degree: 5
>>>> Creating Variables
>>>> Building Constraints
>>>>> using loose objective definition
>>>> Setting Objective
>>>> Solving system
Parameter threads unchanged
   Value: 12  Min: 0  Max: 1024  Default: 0
Parameter MIPFocus unchanged
   Value: 1  Min: 0  Max: 3  Default: 0
>>>> Status Optimal
Solution Time: 0.121


In [8]:
graph = InputDataProcessor.process('data/gc_20_1')
color_picker = ColorPicker(graph, tight_max=True)
solution = color_picker.solve(solver_name="CBC")
g, filename = OutputDataProcessor.process(solution)
print(f"Solution Time: {color_picker.problem.solutionTime:.3f}")
g.show(filename)

>>>> Max degree: 5
>>>> Creating Variables
>>>> Building Constraints
>>>>> using tight objective definition
>>>> Setting Objective
>>>> Solving system
Parameter threads unchanged
   Value: 12  Min: 0  Max: 1024  Default: 0
Parameter MIPFocus unchanged
   Value: 1  Min: 0  Max: 3  Default: 0
>>>> Status Optimal
Solution Time: 0.200


### Example 3: Using Other Solvers

#### Using CBC

In [9]:
graph = InputDataProcessor.process('data/gc_50_5')
color_picker = ColorPicker(graph, tight_max=False)
solution = color_picker.solve(solver_name="CBC", time_limit=25)
g, filename = OutputDataProcessor.process(solution)
print(f"Solution Time: {color_picker.problem.solutionTime:.3f}")
g.show(filename)

>>>> Max degree: 31
>>>> Creating Variables
>>>> Building Constraints
>>>>> using loose objective definition
>>>> Setting Objective
>>>> Solving system
Parameter threads unchanged
   Value: 12  Min: 0  Max: 1024  Default: 0
Parameter MIPFocus unchanged
   Value: 1  Min: 0  Max: 3  Default: 0
>>>> Status Optimal
Solution Time: 21.564


In [10]:
color_count = len(set(solution.colored_nodes.values()))
print(f"Color count: {color_count}")

Color count: 15


#### Using Gurobi

In [11]:
graph = InputDataProcessor.process('data/gc_50_5')
color_picker = ColorPicker(graph, tight_max=False)
solution = color_picker.solve(solver_name="GUROBI", time_limit=25)
g, filename = OutputDataProcessor.process(solution)
print(f"Solution Time: {color_picker.problem.solutionTime:.3f}")
g.show(filename)

>>>> Max degree: 31
>>>> Creating Variables
>>>> Building Constraints
>>>>> using loose objective definition
>>>> Setting Objective
>>>> Solving system
Parameter threads unchanged
   Value: 12  Min: 0  Max: 1024  Default: 0
Parameter MIPFocus unchanged
   Value: 1  Min: 0  Max: 3  Default: 0
Changed value of parameter TimeLimit to 25.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Changed value of parameter MIPGap to 1e-05
   Prev: 0.0001  Min: 0.0  Max: inf  Default: 0.0001
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (linux64)
Thread count: 8 physical cores, 16 logical processors, using up to 12 threads
Optimize a model with 19044 rows, 1601 columns and 41088 nonzeros
Model fingerprint: 0x428873e9
Variable types: 0 continuous, 1601 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic sol

In [12]:
color_count = len(set(solution.colored_nodes.values()))
print(f"Color count: {color_count}")

Color count: 9


# Formulation Issues

In [13]:
graph = InputDataProcessor.process('data/gc_4_4_inf')
color_picker = ColorPicker(graph, neighbors_constr_infeasible=True)
solution = color_picker.solve(solver_name="GUROBI")
g, filename = OutputDataProcessor.process(solution)
print(f"Solution Time: {color_picker.problem.solutionTime:.3f}")
g.show(filename)

>>>> Max degree: 2
>>>> Creating Variables
>>>> Building Constraints
>>>>> using tight objective definition
>>>> Setting Objective
>>>> Solving system
Parameter threads unchanged
   Value: 12  Min: 0  Max: 1024  Default: 0
Parameter MIPFocus unchanged
   Value: 1  Min: 0  Max: 3  Default: 0
Changed value of parameter MIPGap to 1e-05
   Prev: 0.0001  Min: 0.0  Max: inf  Default: 0.0001
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (linux64)
Thread count: 8 physical cores, 16 logical processors, using up to 12 threads
Optimize a model with 28 rows, 13 columns and 68 nonzeros
Model fingerprint: 0xa41f4ddd
Variable types: 0 continuous, 13 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 20 rows and 0 columns
Presolve time: 0.00s
Presolved: 8 rows, 13 columns, 30 nonzeros
Variable types: 0 c

In [14]:
if color_picker.problem.solverModel.Status in [4, 3]:
    color_picker.problem.solverModel.computeIIS()
    color_picker.problem.solverModel.write(f"why_infeasible.ilp")


Computing Irreducible Inconsistent Subsystem (IIS)...

           Constraints          |            Bounds           |  Runtime
      Min       Max     Guess   |   Min       Max     Guess   |
--------------------------------------------------------------------------
        0        28         -         0        25         -           0s
       13        13         -         3         3         -           0s

IIS computed: 13 constraints, 3 bounds
IIS runtime: 0.00 seconds
